In [ ]:
!pip install langchain
!pip install uuid
!pip install pydantic
!pip install langchain_community
!pip install transformers

In [1]:
from typing import List

from langchain_core.tools import StructuredTool

def search_train_ticket(
        origin: str,
        destination: str,
        date: str,
        departure_time_start: str,
        departure_time_end: str
) -> List[dict[str, str]]:
    """按指定条件查询火车票"""

    # mock train list
    return [
        {
            "train_number": "G1234",
            "origin": "北京",
            "destination": "上海",
            "departure_time": "2024-06-01 8:00",
            "arrival_time": "2024-06-01 12:00",
            "price": "100.00",
            "seat_type": "商务座",
        },
        {
            "train_number": "G5678",
            "origin": "北京",
            "destination": "上海",
            "departure_time": "2024-06-01 18:30",
            "arrival_time": "2024-06-01 22:30",
            "price": "100.00",
            "seat_type": "商务座",
        },
        {
            "train_number": "G9012",
            "origin": "北京",
            "destination": "上海",
            "departure_time": "2024-06-01 19:00",
            "arrival_time": "2024-06-01 23:00",
            "price": "100.00",
            "seat_type": "商务座",
        }
    ]


def purchase_train_ticket(
        train_number: str,
) -> dict:
    """购买火车票"""
    return {
        "result": "success",
        "message": "购买成功",
        "data": {
            "train_number": "G1234",
            "seat_type": "商务座",
            "seat_number": "7-17A"
        }
    }


search_train_ticket_tool = StructuredTool.from_function(
    func=search_train_ticket,
    name="查询火车票",
    description="查询指定日期可用的火车票。",
)

purchase_train_ticket_tool = StructuredTool.from_function(
    func=purchase_train_ticket,
    name="购买火车票",
    description="购买火车票。会返回购买结果(result), 和座位号(seat_number)",
)

finish_placeholder = StructuredTool.from_function(
    func=lambda: None,
    name="FINISH",
    description="用于表示任务完成的占位符工具"
)

tools = [search_train_ticket_tool, purchase_train_ticket_tool, finish_placeholder]

In [3]:
from cmllm import TaskDispatcher
from langchain_community.llms.moonshot import Moonshot
import os
# Test chain without prompt
os.environ["MOONSHOT_API_KEY"] = "sk-jp2VoAvlcR3QS8azmXmFIRqxLA4nEVF4o48j8EkJilkm3DfV"
llm = Moonshot(
    temperature=0.2,
    timeout=30
    ) 
my_agent = TaskDispatcher(llm=llm, tools=tools)
task = "我口干舌燥是什么病"
reply = my_agent.run(task)

Success
>>>>Round: 0<<<<



{
  "name": "查询相关资料",
  "args": {
    "query": "口干舌燥 症状 疾病"
  }
}
----
Observation:
没有找到工具


/opt/conda/envs/cmllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>>>Round: 1<<<<

{
  "name": "查询相关资料",
  "args": {
    "query": "口干舌燥 症状 疾病"
  }
}
----
Observation:
没有找到工具
>>>>Round: 2<<<<

Task：你收到的需要执行的任务
- 用户询问“口干舌燥是什么病”，需要查询口干舌燥这一症状可能对应的疾病。

Thinking: 观察你的任务和执行记录，并思考你下一步应该采取的行动
- 根据历史记录，已经尝试过两次查询“口干舌燥 症状 疾病”的相关资料，但都没有找到合适的工具。
- 考虑到中医对症状的诊断可能需要更专业的知识，我将尝试调用一个中医知识库查询工具，以获取更准确的信息。

Action: 调用中医知识库查询工具
```json
{
  "name": "查询相关资料",
  "args": {
    "query": "口干舌燥 症状 中医诊断"
  }
}
```
----
Observation:
没有找到工具
>>>>Round: 3<<<<


KeyboardInterrupt: 

In [5]:
from langchain_community.llms.moonshot import Moonshot
import os
# Test chain without prompt
os.environ["MOONSHOT_API_KEY"] = "sk-jp2VoAvlcR3QS8azmXmFIRqxLA4nEVF4o48j8EkJilkm3DfV"
llm = Moonshot(
    temperature=0.2,
    timeout=30
    ) 

In [13]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    Title: str = Field(description="The title of the movie")
    Director: str = Field(description="The director of the movie")
    Year: int = Field(description="The release year of the movie")


In [14]:
from langchain.output_parsers import PydanticOutputParser

# 定义解析器
parser = PydanticOutputParser(pydantic_object=Movie)

In [15]:
prompt = """
请提供一个电影的详细信息，包括以下格式，变成可供处理的Json：
- Title: 电影标题
- Director: 导演
- Year: 发行年份

例如：
Title: Inception
Director: Christopher Nolan
Year: 2010
"""

In [16]:
# 构建chain以及最后格式化输出
llm_output = llm.invoke(prompt)
print(llm_output)
movie = parser.parse(llm_output)
print(movie)
 

{
  "Title": "Inception",
  "Director": "Christopher Nolan",
  "Year": 2010
}
Title='Inception' Director='Christopher Nolan' Year=2010


In [17]:
from langchain_core.output_parsers import JsonOutputParser
# 构建chain以及最后格式化输出
chain = prompt | llm | JsonOutputParser()
print(chain.invoke({"input": "what's thirteen times 4"}))

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>